# Source From https://www.youtube.com/watch?v=efR1C6CvhmE

- In 1 sentence, SVM use kernal function to transform dataset to higher dimension and draw a hyperplane to do the maximum magin.

In [2]:
# The example from scikit learn doc
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn import svm
from sklearn.datasets import make_blobs

# X : array of shape [n_samples, n_features]

#     The generated samples.  
# y : array of shape [n_samples]

#     The integer labels for cluster membership of each sample. 
X, y = make_blobs(n_samples=40, centers=2, random_state=6)



array([1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1])